# Day 3 - Conversational AI - aka Chatbot!

In [16]:
# imports

import os
from dotenv import load_dotenv
import gradio as gr
from get_api_key import get_api_key

openai, anthropic, gemini = get_api_key()

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI


In [17]:
# Initialize
MODEL = 'gpt-4.1-mini'

In [21]:
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant and for every response, you should say '지민이 사랑해' at the end"

## And now, writing a new callback

We now need to write a function called:

`chat(message, history)`

Which will be a callback function we will give gradio.

### The job of this function

Take a message, take the prior conversation, and return the response.


In [6]:
def chat(message, history):
    return "지민이 사랑해!"

In [8]:
gr.ChatInterface(fn=chat, type="messages").launch(share="true")

* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://bce33e2d9eb4e519d2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


## OK! Let's write a slightly better chat callback!

In [ ]:

def chat(message, history):
    #NOTE: history에 저장된 데이터는 모두 배제하고 role과 content만 남김
    history = [{"role":h["role"], "content":h["content"]} for h in history] #role과 content를 제외한 다른 정보는 모두 배제 - openai 양식과 정확히 같아짐
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content


In [24]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7879
* Running on public URL: https://3bd462c0cb48dcace6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [25]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    #NOTE: stream=True - 데이터를 조금씩 더해가며 덮어쓰기
    #NOTE: yield - 기존 답변에 조금씩 더해가며 덮어쓰기
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [26]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting" again

In [32]:
def chat2(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message_2}] + history + [{"role": "user", "content": message}]
    print(messages)
    #NOTE: stream=True - 데이터를 조금씩 더해가며 덮어쓰기
    #NOTE: yield - 기존 답변에 조금씩 더해가며 덮어쓰기
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

system_message_2 = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [33]:
gr.ChatInterface(fn=chat2, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': '안녕'}]
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': '안녕'}, {'ro

In [34]:
system_message_2 += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [35]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.


In [ ]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    #NOTE: 메시지에 'belt'이 포함되어 있으면 시스템 메시지에 추가 (RAG - Retrieval Augmented Generation)
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>